# Data C182 Final Project

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')
# IMPORTANT: change to your own root folder path! This dir should contain the contents of:
# https://github.com/datac182fa24/final_proj_faulty_commit_student
root_folder = "/content/drive/MyDrive/teaching/data_c182_fa2024/final_project/final_proj_faulty_commit_student/"
import os
os.chdir(root_folder)
# validate that we're in the right directory (ls should show files like: train.py, README.md, etc)
!pwd
!ls

In [ ]:
# Important: enable autoreload so that changes to .py files are auto-imported. For convenience.
#   https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
import seaborn
seaborn.set()

from tqdm.notebook import trange, tqdm
import torch
import os

In [ ]:
# Initialize submission data structure
# `auto_grader_data`: your notebook will populate this dict with important outputs that the 
#   autograder will check.
# IMPORTANT: do not add additional read/write calls to `auto_grader_data` in your own code.
#   Leave the existing provided read/write calls untouched.
#   Violations will be flagged as academic dishonesty.
# WARNING: running this cell will clear the state in `auto_grader_data`. If you run this
#   cell, you'll have to run the rest of the notebook.
from utils.utils import add_entry_to_auto_grader_data
auto_grader_data = {}

# Part 1: Data preprocessing

In this part, you will implement the dataloader and data preprocessors. See the project spec for detailed instructions.

## (1.A) Implement `compute_data_preprocessor()`

In `dataloader/fault_csv_dataset.py`, you will implement the `compute_data_preprocessor()` function. See the project spec for detailed instructions.

Once you've implemented this function correctly, the following cell should succeed:

In [ ]:
from tests.dataloader.test_fault_csv_dataset import TestComputeDataPreprocessor

testcase = TestComputeDataPreprocessor()
testcase.test_simple()
print("Passed tests!")

## (1.B) Implement `FaultCSVDataset`.

Once you've correctly implemented this, the following test case should succeed:

In [ ]:
from tests.dataloader.test_fault_csv_dataset import TestFaultCsvDataset

testcase = TestFaultCsvDataset()
testcase.test_simple()
print("Passed tests!")

## (1.C) Benchmark your Dataloader

We will benchmark your dataloader. To receive full credit, you must achieve a throughput of at least `5000 rows/sec`. The below cell should output something like:
```
features_preproc shape=torch.Size([43505, 1098])
Created FaultCSVDataset (1.4087190628051758 secs)
benchmark_stats: {'throughput_rows_per_sec': 33911.9315827453, 'total_time_secs': 0.11795258522033691, 'latency_secs_avg': 0.00011732101440429687, 'latency_secs_std': 3.2681162353353425e-05}
[1/10877] row_dict={'features': tensor([[-0.3203,  0.0000, -0.0354,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0349,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0356,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0348,  ...,  0.0000,  0.0000,  0.0000]]), 'label': tensor([0, 0, 0, 0])}
[2/10877] row_dict={'features': tensor([[ 3.1216,  0.0000, -0.0101,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0354,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0351,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0353,  ...,  0.0000,  0.0000,  0.0000]]), 'label': tensor([0, 0, 0, 0])}
[3/10877] row_dict={'features': tensor([[-0.3203,  0.0000, -0.0348,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0351,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0354,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0353,  ...,  0.0000,  0.0000,  0.0000]]), 'label': tensor([0, 0, 0, 0])}
[4/10877] row_dict={'features': tensor([[-0.3203,  0.0000, -0.0337,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0272,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0233,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3203,  0.0000, -0.0356,  ...,  0.0000,  0.0000,  0.0000]]), 'label': tensor([0, 0, 0, 0])}
Your dataloader throughput was: 33911.9315827453
Is larger than 5000 rows/sec? True
```

In [ ]:
from tools.benchmark_dataloader import main_benchmark_dataloader
benchmark_stats_pt1c = main_benchmark_dataloader()
print(f"Your dataloader throughput was: {benchmark_stats_pt1c['throughput_rows_per_sec']}")
print(f"Is larger than 5000 rows/sec? {benchmark_stats_pt1c['throughput_rows_per_sec'] >= 5000}")
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_1_c", "dataloader_throughput_rows_per_sec"], benchmark_stats_pt1c["throughput_rows_per_sec"])

# Part 2: Modeling
In this section, you will train several models on the training dataset, using the dataloader you've implemented.

All models will implement the `FaultyCommitClassifierModel` interface (modeling.model_interface.py)

## (2.A) SingleLayerNN
As a warmup, implement the `SingleLayerNN` class (modeling/single_layer_nn.py). This is essentially implementing logistic regression as a NN. Read the class documentation for more information.

If you've implemented it correctly, the following test should pass:

In [ ]:
from tests.modeling.test_single_layer_nn import TestSingleLayerNN

testcase = TestSingleLayerNN()
testcase.test_simple()
print("Test passed!")

## (2.B) Implement `ClassificationTrainer`
Next, implement the `ClassificationTrainer` (in: trainer/trainer.py). Notably, this class contains the training loop, which you will implement.

Tip: feel free to use the training loops from previous homework assignments as reference.

## (2.C) Train a SingleLayerNN
To demonstrate that your dataloader + train loop is working correctly, you will run the `train_and_eval()` function (in train.py). Run the following cell, it should output something like:
```
Train Epoch=4/5 Batch=0/170 loss=0.1745077669620514 train_accuracy=0.9453125 train_accuracy_pos_class=0.20000001788139343                                     | 0/170 [00:00<?, ?it/s] 
GPU max_memory_allocated: 19.313664 MB
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170/170 [00:00<00:00, 351.24it/s] 
Skipping validation (self.skip_val=True)████████████████████████████████████████████████████████████████████████████████▎                         | 139/170 [00:00<00:00, 357.00it/s] 
(val) epoch=5/5: AP=0.00000 (T=1.00000) precision@T=0.00000 recall@T=0.00000 f1@T=0.00000
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.89it/s] 
Finished training! 2.652967691421509 secs (total_num_epochs=5)
Skipping test (skip_test=True)
```
To get full credit, your implementation should get a final train loss less than 0.35.

In [ ]:
from train import main_train_pt2c

train_meta_pt2c, fig_pt2c = main_train_pt2c()
print(f"Final train loss: {train_meta_pt2c.losses[-1]}")
print(f"Is less than 0.35? {train_meta_pt2c.losses[-1] <= 0.35}")
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_2_c", "single_layer_nn_train_loss"], train_meta_pt2c.losses[-1])

# Part 3: Offline Evaluation

In this section, you will implement the offline evaluation pipeline for the binary classification task.

## (3.A) Implement `predict_samples()`
Implement the `predict_samples()` function (in: evaluation/offline_eval.py). This function takes a model, generates model predictions on an evaluation dataset, and stores the inference results into a struct `PredictionMetadata`.

To test your implementation, run the following cell:

In [ ]:
from tests.evaluation.test_offline_eval import TestPredictSamples

testcase = TestPredictSamples()
testcase.test_batchsize_1()
testcase.test_batchsize_3()
testcase.test_batchsize_4()
print("Tests passed!")

## (3.B) Implement `compute_eval_metrics()`
Next, implement the `compute_eval_metrics()` function (in: evaluation/offline_eval.py). Given model predictions and ground truth labels (aka the output of `predict_samples()`), this function will calculate the evaluation metrics that we care about.

To test your implementation, run the following cell:

In [ ]:
from tests.evaluation.test_offline_eval import TestComputeEvalMetrics
testcase = TestComputeEvalMetrics()
testcase.test_simple()
print("Tests passed!")

## (3.C) Evaluate trivial baselines
Implement the above three trivial models (`AlwaysPositiveBinaryClassifier`, `AlwaysNegativeBinaryClassifier`, `RandomBinaryClassifier`), and run the corresponding notebook to test your implementations. Your implementation should output something extremely close to:

<details>
<summary> Click me for desired output </summary>
```
Loaded train dataset from data/split/faulty_commit_dev_train.csv, num_train_rows=43505 dim_input_feats: 1098
======== Test metrics (RandomBinaryClassifier) ========
100%|██████████| 189/189 [00:00<00:00, 379.87it/s]
(Test) AP=0.01375 (T=1.00000) precision@T=0.01386 recall@T=0.50379 f1@T=0.02697
features_preproc shape=torch.Size([43505, 1098])
features_preproc shape=torch.Size([29003, 1098])
features_preproc shape=torch.Size([48337, 1098])
Loaded train dataset from data/split/faulty_commit_dev_train.csv, num_train_rows=43505 dim_input_feats: 1098
======== Test metrics (AlwaysPositiveBinaryClassifier) ========
100%|██████████| 189/189 [00:00<00:00, 236.68it/s]
(Test) AP=0.01363 (T=1.00000) precision@T=0.01363 recall@T=1.00000 f1@T=0.02690
======== Test metrics (AlwaysNegativeBinaryClassifier) ========
100%|██████████| 189/189 [00:00<00:00, 241.45it/s]
(Test) AP=0.01363 (T=0.00000) precision@T=0.01363 recall@T=1.00000 f1@T=0.02690
```
</details>


In [ ]:
from tools.evaluate_trivial_models import main_evaluate_model_random, main_evaluate_always_pos_neg
test_eval_metrics_pt3c_random = main_evaluate_model_random()
test_eval_metrics_pt3c_always_pos, test_eval_metrics_pt3c_always_neg = main_evaluate_always_pos_neg()
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_3_c", "random_binary_classifier", "test_eval_metrics"], test_eval_metrics_pt3c_random.to_pydict())
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_3_c", "always_pos_classifier", "test_eval_metrics"], test_eval_metrics_pt3c_always_pos.to_pydict())
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_3_c", "always_neg_classifier", "test_eval_metrics"], test_eval_metrics_pt3c_always_neg.to_pydict())

# Part 4: Train and eval SingleLayerNN
Once (Part 1), (Part 2), (Part 3) are done, let's put all of the parts together and run the full train+eval pipeline! Run the following notebook cells, it will train a `SingleLayerNN` model on the training dataset, measure eval metrics on the validation dataset after each training epoch, and then evaluate on the test set at the end of training. Then, it will generate various plots (eg: train loss over time, eval metrics on val/test sets).

To receive full credit, you should achieve a test AP of greater than 0.015.

If everything is implemented correctly (dataloading, model architecture, and evaluation pipeline), then you should receive something like this:
```
(val) epoch=10/10: AP=0.05734 (T=0.32974) precision@T=0.11614 recall@T=0.19283 f1@T=0.14496
Finished training! 12.00160551071167 secs (total_num_epochs=10)
======== Validation metrics ========
[epoch=1/10] (val) AP=0.03989 (T=0.48066) precision@T=0.09879 recall@T=0.18089 f1@T=0.12779
[epoch=2/10] (val) AP=0.04018 (T=0.42750) precision@T=0.09364 recall@T=0.18601 f1@T=0.12457
[epoch=3/10] (val) AP=0.04002 (T=0.38345) precision@T=0.09706 recall@T=0.16894 f1@T=0.12329
[epoch=4/10] (val) AP=0.04081 (T=0.36203) precision@T=0.09446 recall@T=0.15700 f1@T=0.11795
[epoch=5/10] (val) AP=0.04321 (T=0.33020) precision@T=0.09735 recall@T=0.16894 f1@T=0.12352
[epoch=6/10] (val) AP=0.04669 (T=0.31828) precision@T=0.10200 recall@T=0.18259 f1@T=0.13089
[epoch=7/10] (val) AP=0.04916 (T=0.29781) precision@T=0.10388 recall@T=0.19625 f1@T=0.13585
[epoch=8/10] (val) AP=0.05088 (T=0.31498) precision@T=0.11187 recall@T=0.19454 f1@T=0.14206
[epoch=9/10] (val) AP=0.05444 (T=0.32641) precision@T=0.11626 recall@T=0.19283 f1@T=0.14506
[epoch=10/10] (val) AP=0.05734 (T=0.32974) precision@T=0.11614 recall@T=0.19283 f1@T=0.14496
======== Test metrics ========
100%|██████████| 48/48 [00:00<00:00, 60.82it/s]
/content/drive/MyDrive/teaching/data_c182_fa2024/final_project/final_proj_faulty_commit_sol/evaluation/offline_eval.py:136: RuntimeWarning: invalid value encountered in divide
  f1s = 2.0 * (precisions * recalls) / (precisions + recalls)
(Test) AP=0.02361 (T=0.60950) precision@T=0.07212 recall@T=0.07891 f1@T=0.07536
Using validation set operating point threshold T=0.3297363519668579:
(Test) (T=0.32974) precision@T=0.03173 recall@T=0.18665 f1@T=0.05424
Saving figure to: submission/main_train_pt4.png
```

In [ ]:
from train import main_train_and_eval_pt4

train_meta_pt4, (test_eval_metrics_pt4, test_metrics_op_pt4), fig_pt4 = main_train_and_eval_pt4()
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_4_a", "single_layer_nn", "train_meta"], train_meta_pt4.to_pydict())
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_4_a", "single_layer_nn", "test_eval_metrics"], test_eval_metrics_pt4.to_pydict())
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_4_a", "single_layer_nn", "test_metrics_op"], test_metrics_op_pt4.to_pydict())
print(f"Test AP {test_eval_metrics_pt4.average_precision:.5f} >= 0.015? {test_eval_metrics_pt4.average_precision >= 0.015}")

# Part 5: Train a better model

Challenge: implement a new model architecture that outperforms the `SingleLayerNN` model you implemented. To achieve full credit, your model must achieve greater than 0.025 test AP.

Tip: implement your model in modeling/, then modify `train_and_eval_pt5()` (in train_pt5.py) to train+eval your new model.

In [ ]:
from train_pt5 import train_and_eval_pt5

train_metadata_pt5, (test_eval_metrics_pt5, test_metrics_op_pt5), fig = train_and_eval_pt5()

print(f"(my_model) Got test AP: {test_eval_metrics_pt5.average_precision}")
print(f"Greater than 0.025? {test_eval_metrics_pt5.average_precision >= 0.025}")

add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_5_a", "model_v2", "train_meta"], train_metadata_pt5.to_pydict())
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_5_a", "model_v2", "test_eval_metrics"], test_eval_metrics_pt5.to_pydict())
add_entry_to_auto_grader_data(auto_grader_data, ["output", "part_5_a", "model_v2", "test_metrics_op"], test_metrics_op_pt5.to_pydict())

# Submission instructions
Run the following cell below to generate + download your submission zip file: datac182_final_proj_submission.zip. You will submit this zip file to Gradescope.

In [ ]:
# IMPORTANT: be sure that `root_folder` is defined correctly in previous cell
from utils.utils import save_auto_grader_data
from consts import STUDENT_SUBMISSION_OUTDIR
import os
outpath_autograder_data = os.path.join(STUDENT_SUBMISSION_OUTDIR, "student_submission_data.pt")
save_auto_grader_data(auto_grader_data, outpath=outpath_autograder_data)
print(f"Saved auto_grader_data to: {outpath_autograder_data}")

os.chdir(root_folder)
!pwd # make sure we are in the right dir
!ls

# Note: It's OK if the `rm` command fails, this is normal if this is your first
#   time running this.
!rm datac182_final_proj_submission.zip
# zip everything (maintain dir structure) but exclude datasets (to keep filesize down)
!zip -r datac182_final_proj_submission.zip . -x "data/*" "__pycache__/*" "*/__pycache__/*" ".git/*"

from google.colab import files
outpath_zip = "datac182_final_proj_submission.zip"
files.download(outpath_zip)
print(f"Finished downloading {outpath_zip}! Upload this zip file to Gradescope as your submission to run the autograder. The {outpath_zip} file will be in your browser's default download directory.")